In [ ]:
!pip install av
!pip install torch torchvision
!pip install opencv-python
!pip install yolov5

import av
import torch
from yolov5 import YOLOv5
import json
import cv2
import numpy as np

print("done")

done


In [ ]:
# Function to extract frames
def extract_frames(video_path, interval=1):
    container = av.open(video_path)
    frames = []
    timestamps = []
    for frame in container.decode(video=0):
        # Extract frames based on the specified interval
        if frame.time >= len(timestamps) * interval:
            frames.append(frame.to_image())
            timestamps.append(frame.time)
    return frames, timestamps


In [ ]:
# Function to detect logos
def detect_logos(model, frames, timestamps):
    pepsi_timestamps = []
    cocacola_timestamps = []

    for i, frame in enumerate(frames):
        frame = np.array(frame)  # Convert PIL image to numpy array
        results = model(frame)  # Get predictions from the model
        predictions = results.pred[0].cpu().numpy()  # Convert predictions to numpy array

        for pred in predictions:
            # The format of pred is [x1, y1, x2, y2, confidence, class]
            class_id = int(pred[5])
            if class_id == 0:  # Assuming class 0 is Pepsi
                pepsi_timestamps.append(timestamps[i])
            elif class_id == 1:  # Assuming class 1 is CocaCola
                cocacola_timestamps.append(timestamps[i])

    return pepsi_timestamps, cocacola_timestamps

In [ ]:
# Function to save results
def save_results(pepsi_timestamps, cocacola_timestamps, output_path='results.json'):
    results = {
        "Pepsi_pts": pepsi_timestamps,
        "CocaCola_pts": cocacola_timestamps
    }
    with open(output_path, 'w') as f:
        json.dump(results, f)


In [ ]:
# Load model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # Download and load pre-trained YOLOv5s model


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-13 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# Extract frames from video
video_path = '/content/Coke_Pepsi_Video.mp4'  # Use the uploaded file path
frames, timestamps = extract_frames(video_path, interval=1)  # Adjust interval as needed


In [ ]:
# Detect logos
pepsi_timestamps, cocacola_timestamps = detect_logos(model, frames, timestamps)


In [ ]:
# Save results to JSON
save_results(pepsi_timestamps, cocacola_timestamps)

print("Logo detection completed and results have been saved to a new file-'results.json'")

Logo detection completed and results have been saved to a new file-'results.json'
